# Implementation of "Rational Criteria for Diagonalizability of Real Matrices"
## Carlos A.C.C. Perello (Imperial College London)
## Original work by João Ferreira Alves (ULisboa IST)

In [239]:
using LinearAlgebra, Polynomials, Nemo

We implement the Frobenius inner product of $A, B$:
$$
\langle A, B\rangle = \text{tr}(AB^*)
$$

In [240]:
function frob_ip(A,B)
    ret = tr(A*B')
    ret
end

frob_ip (generic function with 1 method)

We now implement the matrix $K_p(A)$:
$$
K_p(A) := [\text{tr}(A^{i+j-2})]_{i,j=1}^p = \begin{pmatrix}
\text{tr}(A^0) & \text{tr}(A^1) & \dots & \text{tr}(A^{p-1})\\
\text{tr}(A^1) & \text{tr}(A^2) & \dots & \text{tr}(A^{p})\\
\vdots & & \ddots & \vdots\\
\text{tr}(A^{p-1}) & \text{tr}(A^{p}) & \dots & \text{tr}(A^{2p-2})\\
\end{pmatrix}
$$

And $L_p(A)$:
$$
L_p(A) := \left[\langle A^{i-1}, A^{j-1}\rangle\right]_{i,j=1}^p = \begin{pmatrix}
\text{tr}((A^*)^0) & \text{tr}((A^*)^1) & \dots & \text{tr}((A^*)^{p-1})\\
\text{tr}(A^1(A^*)^0) & \text{tr}(A^1(A^*)1) & \dots & \text{tr}(A^1(A^*)^{p-1})\\
\vdots & & \ddots & \vdots\\
\text{tr}(A^{p-1}(A^*)^0) & \text{tr}((A^{p-1}(A^*)^{1}) & \dots & \text{tr}(A^{p-1}(A^*)^{p-1})\\
\end{pmatrix}
$$

In [291]:
function Kₚ(A, p)
    A = Matrix{BigFloat}(A)
    mat = zeros(p,p)
    row = [A^i for i = 0:p-1]
    for i = 1:p
        mat[i,:] = Array{BigFloat}(tr.(row))
        row = [A*r for r in row]
    end
    mat
end

function inefficient_Kₚ(A, p)
    mat = zeros(p,p)
    for i = 1:p
        for j = 1:p
            mat[i,j] = tr(A^(i+j-2))
        end
    end
    mat
end

function Lₚ(A, p)
    mat = zeros(p, p)
    row = [A'^i for i = 0:p-1]
    for i = 1:p
        mat[i,:] = tr.(row)
        row = [A*r for r in row]
    end
    mat
end

Lₚ (generic function with 1 method)

And now we implement the invariant $r$:
$$
r := \min\{p=1,\dots,n: |K_{p+1}(A)|\leq 0\}
$$

In [320]:
function r_func(A)
    # this is the problem
    A = Matrix{BigFloat}(A)
    n = size(A)[1]
    ret = 0
    for p=1:n
        if det(Kₚ(A, p+1)) ≤ 1e-5 || det(Kₚ(A, p+1)) ≥ 1e-5
            ret = p
        end
    end
    ret
end

test_mat = Diagonal(1:0.5:3.5)
n = size(test_mat)[1]

r_func(test_mat)
# sometimes we see overflow
# In a distrinct diagonal matrix, there are n distinct eigenvalues #
# Why is the determinant of Kₚ(D,10) < 0, when the paper states it shouldn't be the case? 
# Is it due to numerical error / not using BigFloats?

6

Implement Theorem 2.2: if $d$ is the number of distinct eigenvalues of $A$, and $d_\mathbb{R}$ is the number of distinct real eigenvalues of $A$, we have:
$$
d = \text{rank}(K_n(A)), \quad d_\mathbb{R} = \text{signature}(K_n(A))
$$

Moreover, if $A$ is nonsingular and if we define $V(A)$ to be the number of sign variations in the determinants of $K_r(A), \dots, K_d(A)$, we have:
$$
d_{\mathbb{R}} = d - 2V(A)
$$

In [321]:
function fast_d(A)
    n = size(A)[1]
    ret = rank(Kₚ(A, n))
    ret
end

function fast_dr(A)
    ret = sum(eigvals(A) > 0) - sum(eigvals(A) < 0)
    ret
end

function V(A)
    variations = 0
    r = r_func(A)
    d = fast_d(A)
    prev_sgn = sign(det(Kₚ(A,r)))
    for i = (r+1):d
        sgn = sign(det(Kₚ(A,i)))
        if prev_sgn != sgn
            variations = variations + 1
        end
        prev_sgn = sgn
    end
end

function faster_dr(A)
    if det(A) == 0
        DomainError(A, "input must be nonsingular")
    end
    ret = fast_d(A) - 2*V(A)
    ret
end

faster_dr (generic function with 1 method)

Theorem 2.3 is hard to implement, as there are no ways to calculate $m$, the degreen of the minimal polynomial of $A$, using Julia.

We implement Theorem 3.1, which states that a matrix $A$ is diagonalisable $\iff$ $|K_{r+1}(A)| = |L_{r+1}(A)|$, where $r$ is the invariant defined above. We also imlpement the corollary

In [322]:
function is_diag(A)
    r = r_func(A)
    dK = det(Kₚ(A, r+1))
    if dK < 0
        return false
    end
    ret = det(Lₚ(A, r+1)) ≈ dK
    ret
end

is_diag(I(4)) # test

true

We now implement Example 3.3; we first construct the $n\times n$ square matrix:
$$
A = \begin{pmatrix}
1 & 1& \dots & 1 & 1 & 1 & a\\
1 & 1& \dots & 1 & 1 & b & 1\\
\vdots & \vdots& & \vdots & \vdots & \vdots & \vdots \\\
1 & a& \dots & 1 & 1 & 1 & 1\\
b & 1& \dots & 1 & 1 & 1 & 1\\
\end{pmatrix}
$$

In [323]:
function offd_mat(a, b, n)
    if n%2 == 1
        throw(DomainError(n, "n must be even"))
    end
    ret = ones(n, n)
    off_diag = zeros(n)
    for i = 1:n
        if i % 2 == 1
            val = a
        else
            val = b
        end
        ret[i, n-i+1] = val
    end
    ret
end

n=4
is_diag(offd_mat(1,2,n)) # test, should be false

false

We now implement Example 3.4

In [324]:
function non_derag(n)
    d = [3; fill(2, n-1)]
    ld = fill(1, n-1)
    bd = Bidiagonal(d, ld, :L)
    ret = bd + ones(n,n)
    ret
end

n=4
isdiag(non_derag(n)) # should be false

false

We now implement Theorem 3.5, which states that $A\sim B \iff K_{r+1}(A) = K_{r+1}(B)$ 

In [325]:
function is_similar(A, B)
    ra = r_func(A)
    rb = r_func(B)
    ret = Kₚ(A, ra+1) ≈ Kₚ(B, rb+1)
    ret
end


is_similar (generic function with 1 method)

Examples 3.6-3.8 are hard to implement, as they give a criteria for checking similarity to a very specific case of the matrix defined in Example 3.3. This is already implemented above in `is_similar`.

We now turn our attention to section 3.2 and define the polynomial $\psi_A(z)$, defined as:
$$
\psi_A(z) = \begin{cases}
\sum_{i=0}^{r-1} c_iz^i + z^r, \text{ if } |K_{r+1}(A)| = 0\\
1, \text{ else}
\end{cases}
$$

Where $\mathbf{c} = (c_0,\cdots, c_{r-1}, 1)$ denotes the unique vector with unitary last component in the kernel of $K_{r+1}(A)$; the kernel of $K_{r+1}(A)$ is 1-dimensional in this case as $|K_{r+1}(A)|=0$

In [326]:
function ψ(A::Matrix{Any}) # Doesn't work
    A = Matrix(A)
    r = r_func(A)
    Kr_o = Kₚ(A, r+1)
    if abs(det(Kr_o)) > 1e-5
        return Polynomial([1])
    end
    c_bar = vec(nullspace(Kr_o))
    c = c_bar./(last(c_bar))
    ret = Polynomial(c)
    ret
end


ψ (generic function with 2 methods)

And now we implement Theorem 3.11:

In [334]:
function is_diag_poly(A)
    A = Matrix{BigFloat}(A)
    n = size(A)[1]
    poly = ψ(A)
    ret = tr(poly(A)) ≤ 1e-6 || tr(poly(A)) ≥ 1e-6
    ret
end

is_diag_poly (generic function with 1 method)

In [335]:
is_diag_poly(test_mat)

true